In [1]:
import pyspark
from pyspark.sql import SparkSession

### Question 1

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/01 02:44:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [31]:
spark.version

'3.2.1'

In [22]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

--2022-03-01 02:28:08--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 54.231.133.193
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|54.231.133.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-02.csv’



             fhvhv_   9%[>                   ]  66.56M   949KB/s    eta 6m 20s 

2.csv                11%[=>                  ]  82.29M  1.48MB/s    eta 6m 17s 

          fhvhv_tri  12%[=>                  ]  89.57M  1.63MB/s    eta 6m 13s 

 fhvhv_tripdata_202  13%[=>                  ]  94.90M  1.79MB/s    eta 6m 6s  

...

ipdata_2021-02.csv   16%[==>                 ] 118.24M  1.43MB/s    eta 6m 7s  

2021-02.csv          17%[==>                 ] 121.46M  1.55MB/s    eta 6m 5s  

v                    18%[==>                 ] 126.74M  1.78MB/s    eta 6m 1s  

                  f  18%[==>                 ] 127.67M  1.75MB/s    eta 6m 1s  

fhvhv_tripdata_2021 100%[===================>] 699.83M  2.10MB/s    in 7m 20s  

2022-03-01 02:35:29 (1.59 MB/s) - ‘fhvhv_tripdata_2021-02.csv’ saved [733822658/733822658]



In [3]:
!ls -lh fhvhv_tripdata_2021-02.csv

-rw-rw-r-- 1 desenfirman desenfirman 700M Oct 30 01:53 fhvhv_tripdata_2021-02.csv


In [4]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-02.csv')

In [5]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true)))

In [6]:
!head -n 1001 fhvhv_tripdata_2021-02.csv > head_02.csv

In [7]:
import pandas as pd

In [8]:
df_pandas = pd.read_csv('head_02.csv')

In [9]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [44]:
spark.createDataFrame(df_pandas).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

Integer - 4 bytes
Long - 8 bytes

In [10]:
from pyspark.sql import types

In [11]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [12]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-02.csv')

In [13]:
df = df.repartition(24)

In [14]:
df.write.parquet('fhvhv/2021/02/')

In [15]:
df = spark.read.parquet('fhvhv/2021/02/')

In [16]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



### Question 2

In [29]:
! du -h fhvhv

211M	fhvhv/2021/02
211M	fhvhv/2021
211M	fhvhv


### Question 3

In [18]:
from pyspark.sql import functions as F

In [32]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02867|2021-02-03 08:26:16|2021-02-03 08:31:19|         171|         171|   null|
|           HV0003|              B02617|2021-02-03 09:34:52|2021-02-03 09:56:39|         181|         211|   null|
|           HV0003|              B02878|2021-02-01 06:15:00|2021-02-01 06:53:20|         155|         107|   null|
|           HV0003|              B02883|2021-02-03 10:20:57|2021-02-03 10:46:25|         213|         247|   null|
|           HV0005|              B02510|2021-02-01 17:40:56|2021-02-01 18:08:37|         189|          21|   null|
|           HV0003|              B02682|2021-02-03 10:36:25|2021-02-03 11:10:10|

In [24]:
df.registerTempTable('fhvhv_tripdata_2021_02')

/home/desenfirman/miniconda3/lib/python3.8/site-packages/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


In [26]:
spark.sql("""
SELECT
    *
FROM
    fhvhv_tripdata_2021_02
WHERE
    pickup_datetime >= '2021-02-15 00:00:00' and pickup_datetime <= '2021-02-15 23:59:59'
""").count()

367170

### Question 4

In [37]:
spark.sql("""
WITH a AS (
    SELECT
        *,
        dropoff_datetime - pickup_datetime as duration
    FROM
        fhvhv_tripdata_2021_02
)
    SELECT 
        DATE_TRUNC('day', pickup_datetime) as pickup_date,
        MAX(duration) as max_duration
    FROM a
    GROUP BY pickup_date
    ORDER BY 2 DESC
""").show()

+-------------------+--------------------+
|        pickup_date|        max_duration|
+-------------------+--------------------+
|2021-02-11 00:00:00|INTERVAL '0 20:59...|
|2021-02-17 00:00:00|INTERVAL '0 15:53...|
|2021-02-20 00:00:00|INTERVAL '0 12:13...|
|2021-02-03 00:00:00|INTERVAL '0 11:17...|
|2021-02-19 00:00:00|INTERVAL '0 10:26...|
|2021-02-25 00:00:00|INTERVAL '0 09:43...|
|2021-02-18 00:00:00|INTERVAL '0 09:36...|
|2021-02-10 00:00:00|INTERVAL '0 09:29...|
|2021-02-21 00:00:00|INTERVAL '0 08:57...|
|2021-02-09 00:00:00|INTERVAL '0 08:54...|
|2021-02-06 00:00:00|INTERVAL '0 08:44...|
|2021-02-02 00:00:00|INTERVAL '0 08:35...|
|2021-02-05 00:00:00|INTERVAL '0 08:28...|
|2021-02-12 00:00:00|INTERVAL '0 08:22...|
|2021-02-08 00:00:00|INTERVAL '0 08:21...|
|2021-02-14 00:00:00|INTERVAL '0 08:16...|
|2021-02-22 00:00:00|INTERVAL '0 07:51...|
|2021-02-27 00:00:00|INTERVAL '0 07:32...|
|2021-02-15 00:00:00|INTERVAL '0 07:11...|
|2021-02-04 00:00:00|INTERVAL '0 07:06...|
+----------

### Question 5

In [38]:
spark.sql("""
    SELECT
        dispatching_base_num,
        count(dispatching_base_num) as num_occurrences
    FROM
        fhvhv_tripdata_2021_02
    GROUP BY dispatching_base_num
    ORDER BY 2 DESC
""").show()

+--------------------+---------------+
|dispatching_base_num|num_occurrences|
+--------------------+---------------+
|              B02510|        3233664|
|              B02764|         965568|
|              B02872|         882689|
|              B02875|         685390|
|              B02765|         559768|
|              B02869|         429720|
|              B02887|         322331|
|              B02871|         312364|
|              B02864|         311603|
|              B02866|         311089|
|              B02878|         305185|
|              B02682|         303255|
|              B02617|         274510|
|              B02883|         251617|
|              B02884|         244963|
|              B02882|         232173|
|              B02876|         215693|
|              B02879|         210137|
|              B02867|         200530|
|              B02877|         198938|
+--------------------+---------------+
only showing top 20 rows



### Question 6 

Read zones table and create spark table for Spark SQL Purposes

In [39]:
zones = spark.read.parquet('zones/')
zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [40]:
zones.registerTempTable('zones_data')

/home/desenfirman/miniconda3/lib/python3.8/site-packages/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


In [43]:
spark.sql("""
    WITH a AS (
        SELECT
            fhvhv_tripdata_2021_02.*,
            pu_zones.Zone as pu_zone,
            do_zones.Zone as do_zone
        FROM
            fhvhv_tripdata_2021_02
        LEFT JOIN zones_data pu_zones
            ON pu_zones.LocationID = fhvhv_tripdata_2021_02.PULocationID
        LEFT JOIN zones_data do_zones
            ON do_zones.LocationID = fhvhv_tripdata_2021_02.DOLocationID
    )
    SELECT 
        CONCAT(pu_zone, '/', do_zone) as zone_pair,
        COUNT(*) as num_occurrences
    FROM a
    GROUP BY pu_zone, do_zone
    ORDER BY 2 DESC
""").show()

+--------------------+---------------+
|           zone_pair|num_occurrences|
+--------------------+---------------+
|East New York/Eas...|          45041|
|Borough Park/Boro...|          37329|
|   Canarsie/Canarsie|          28026|
|Crown Heights Nor...|          25976|
| Bay Ridge/Bay Ridge|          17934|
|Jackson Heights/J...|          14688|
|     Astoria/Astoria|          14688|
|Central Harlem No...|          14481|
|Bushwick South/Bu...|          14424|
|Flatbush/Ditmas P...|          13976|
|South Ozone Park/...|          13716|
|Brownsville/Brown...|          12829|
|      JFK Airport/NA|          12542|
|Prospect-Lefferts...|          11814|
|Forest Hills/Fore...|          11548|
|Bushwick North/Bu...|          11491|
|Bushwick South/Bu...|          11487|
|Crown Heights Nor...|          11462|
|Crown Heights Nor...|          11342|
|Prospect-Lefferts...|          11308|
+--------------------+---------------+
only showing top 20 rows

